Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

import camb
from camb import model, initialpower

In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'

In [4]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [5]:
#Fiducial cosmological parameters
c=3e5
hubble=0.675
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9

nss = 0.9655
alphass = 0.0
betass = 0.0

gamma=0.545

kpivot = 0.05 

In [6]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass, nrun=alphass, nrunrun=betass, pivot_scalar=kpivot)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [7]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [8]:
#Get matter power spectrum at z=0: P(k,z=0)
#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)
#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-5, maxkh=1.0, npoints = 1600)

In [9]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [10]:
#Derivatives

def dlogPd_dns(kk):
    return np.log(kk/kpivot)

def dlogPd_dalphas(kk):
    return (1./2)*pow(np.log(kk/kpivot),2)

def dlogPd_dbetas(kk):
    return (1./6)*pow(np.log(kk/kpivot),3)

In [11]:
#Redshift bins for IM interferometer
zlist = np.arange(3.05,5.0,0.1)
ztest = zlist[10]
Nzbins = len(zlist)

print zlist
print "ztest =", ztest
print "Number of redshift bins =", Nzbins

[ 3.05  3.15  3.25  3.35  3.45  3.55  3.65  3.75  3.85  3.95  4.05  4.15
  4.25  4.35  4.45  4.55  4.65  4.75  4.85  4.95]
ztest = 4.05
Number of redshift bins = 20


In [12]:
midfreq = 1420.4e6 #Hz

#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00
#Define the comoving distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

def ynu(zc):
    return c*pow(1+zc,2)/(midfreq*Hz(zc))

print ztest, rcom(ztest), DA(ztest)

4.05 7403.32375276 1466.00470352


In [13]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.990681848932


In [14]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest)

0.251144113086


In [15]:
#fiducial bHI from Bull et al 2015
def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

#fiducial OmHI Mario's fit
def OmHI(zc):
    return 0.00048+0.00039*zc-0.000065*pow(zc,2)

print bHI(ztest), OmHI(ztest)

2.219125 0.0009933375


In [16]:
#Construct P(k,μ,z) 
def beta(zc):
    return fg(zc)/bHI(zc)

def Pkz(kk,mu,zc):    
    return pow(1+beta(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [18]:
#mean brightness temperature [mK] Mario's fit
def Tb(zc):
    return 0.0559+0.2324*zc-0.024*pow(zc,2)

print ztest, Tb(ztest)

4.05 0.60346


In [19]:
#Construct P_HI(k,μ,z), Tb assumed known
def PHI(kk,mu,zc):
    return pow(bHI(zc),2)*Pkz(kk,mu,zc)

In [20]:
def thetab(zc):
    return 21*(1+zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

def lam(zc):
    return 21*(1+zc)

In [23]:
#Interferometer noise specs

def Tsys(zc):
    Tinst = 15.0*1e3 #mK
    fc = 1420.4/(1+zc)
    Tsky = 60*pow(300/fc,2.55)*1e3 #mK
    return Tinst + Tsky 

Nd = 7000 #number of dishes
Ddish = 6.0*100 #cm 
Dmax = 700.0*100 #max baseline in cm

def FOV(zc):
    return pow(lam(zc)/Ddish,2)

fsky=0.5
Area=41253*fsky

omegatot = Area*pow(pi/180,2)
ttotal = 10000*60*60 #sec

Dzbin = 0.1

#filling factor check (has to be <1)
ff = Nd*(Ddish/Dmax)**2
print "fcover =", ff

#noise PS
def Pnoise(kk,zc):
    eta = 1.0 #efficiency
    Ae = eta*pi*(Ddish/2)**2
    nu = (Nd*(Nd-1))*lam(zc)**2/(2*pi*(Dmax**2-Ddish**2))
    np = 2
    #note division with Tb^2
    return (1/pow(Tb(zc),2))*(lam(zc)**4*rcom(zc)**2*ynu(zc)*Tsys(zc)**2*omegatot)/(np*Ae**2*FOV(zc)*nu*ttotal) 

fcover = 0.514285714286


In [48]:
def kmin(zc): 
    return 2*pi*Ddish/(rcom(zc)*lam(zc))

def kmax_int(zc): #instrumental cutoff
    return 2*pi*Dmax/(rcom(zc)*lam(zc))

def kmax(zc): #non-linear cutoff (Smith et al 2003)
    kNL = 0.14
    return kNL*pow(1+zc,2/(2+nss))

for zi in range(0,Nzbins):
    zc = zlist[zi]
    print zc, kmin(zc), kmax(zc), kmax_int(zc)

3.05 0.00673242684371 0.359591474608 0.785449798433
3.15 0.00647701520178 0.365555728583 0.755651773541
3.25 0.0062391735324 0.371473369392 0.72790357878
3.35 0.0060171969987 0.377345845615 0.702006316515
3.45 0.00580959092324 0.383174528663 0.677785607711
3.55 0.00561503972451 0.388960718516 0.65508796786
3.65 0.0054323811606 0.394705648934 0.63377780207
3.75 0.0052605848581 0.400410492178 0.613734900111
3.85 0.00509873432207 0.406076363294 0.594852337575
3.95 0.00494601178868 0.411704324035 0.57703470868
4.05 0.00480168542008 0.417295386423 0.560196632343
4.15 0.00466509841624 0.422850516016 0.544261481895
4.25 0.00453565973847 0.4283706349 0.529160302821
4.35 0.00441283616133 0.433856624437 0.514830885488
4.45 0.00429614544237 0.439309327786 0.501216968276
4.55 0.00418515043076 0.444729552227 0.488267550255
4.65 0.00407945396852 0.450118071311 0.475936296328
4.75 0.00397869446336 0.455475626831 0.464181020725
4.85 0.00388254203263 0.460802930664 0.45296323714
4.95 0.0037906951349 0.

In [49]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

#effective volume going in the Fisher matrix
def Veff(kk,mu,zc):
    return Vsur(zc)*(PHI(kk,mu,zc)/(PHI(kk,mu,zc)+Pnoise(kk,zc)))**2

print "%.4g" % Vsur(ztest)

2.404e+10


In [50]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [51]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [52]:
mugrid = np.linspace(-1., 1., 513) 

In [53]:
#Fisher matrix parameters 
params = ["0:ns","1:alphas","2:betas"]

In [54]:
#%%time
#   Fisher matrix   # 

Npar = 2 #Npar = 3
#create array of zeros
s = Npar

Fishermat = np.zeros([s,s])

for zi in range(0,Nzbins):
    zc = zlist[zi]
    if kmax_int(zc) < kmax(zc): kmaxx = kmax_int(zc)
    else: kmaxx = kmax(zc)    
    kgrid = np.linspace(kmin(zc), kmaxx, 513)
    if kpivot < kmin(zc) or kpivot > kmaxx: raise ValueError("kpivot out of range")
    K, MU = np.meshgrid(kgrid, mugrid)
    for i in range(0,Npar):  
        def deriv_i(kk,mu,zc):
            if i==0:  return dlogPd_dns(kk)
            elif i==1:  return dlogPd_dalphas(kk)
            elif i==2:  return dlogPd_dbetas(kk)
            else: print "out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlogPd_dns(kk)
                    elif j==1:  return dlogPd_dalphas(kk)
                    elif j==2:  return dlogPd_dbetas(kk)
                    else: print "index out of range" 
                Fishermat[i][j] += integrate2D(dF(K,MU),kgrid,mugrid)
            else: Fishermat[i,j] = Fishermat[j,i]

In [55]:
print Fishermat

[[ 75360085.43917032  65891936.08784308]
 [ 65891936.08784308  59332632.61173631]]


In [56]:
print linalg.inv(Fishermat)

[[  4.57937118e-07  -5.08562692e-07]
 [ -5.08562692e-07   5.81639122e-07]]


In [57]:
#print marginalised uncertainties on (ns,alphas,betas)
for i in range(0,Npar):
    print np.sqrt(linalg.inv(Fishermat)[i,i])

0.000676710512991
0.000762652687991
